In [ ]:
from os import getenv, path
from pathlib import Path
input_folder = Path(getenv(
    'CROSSCOMPUTE_INPUT_FOLDER', 'batches/standard/input'))
output_folder = Path(getenv(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'batches/standard/output'))

In [1]:
import json

parameters = json.load(open(input_folder / 'variables.dictionary', 'rt'))    

fill_color = parameters.get('fill_color', 'black')
back_color = parameters.get('back_color', 'white')
box_size = parameters.get('box_size', 10)
border = parameters.get('border', 4)
logo_path = input_folder / 'logo.png'
rounded_corner_radius = parameters.get('rounded_corner_radius', 10)
rounded_corner_color = parameters.get('rounded_corner_color', 'black')
style = parameters.get('style', 'rounded')

# Network parameters
network_name = parameters.get('network_name', '')
hidden = parameters.get('hidden', '')
password = getenv('password', '')
encryption = parameters.get('encryption', '')

In [1]:
hidden_str = 'TRUE' if hidden else 'FALSE'
data = f'WIFI:S:{network_name};T:{encryption};P:{password};H:{encryption};;'

NameError: name 'input_folder' is not defined

In [81]:
import qrcode
from PIL import Image, ImageDraw
from qrcode.image.styledpil import StyledPilImage
from qrcode.image.styles.moduledrawers import RoundedModuleDrawer, SquareModuleDrawer, GappedSquareModuleDrawer, CircleModuleDrawer, VerticalBarsDrawer, HorizontalBarsDrawer
from qrcode.image.styles.colormasks import SolidFillColorMask

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')  # Remove '#' if present
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return r, g, b



style_modules = {
    "rounded": RoundedModuleDrawer, 
    "square": SquareModuleDrawer, 
    "gapped-square": GappedSquareModuleDrawer, 
    "circle": CircleModuleDrawer, 
    "vertical": VerticalBarsDrawer, 
    "horizontal": HorizontalBarsDrawer
}

# Generate QR code
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_Q,
    box_size=box_size,
    border=border,
)
qr.add_data(data)
qr.make(fit=True)
qr_img = qr.make_image(fill_color=fill_color, 
                       back_color=back_color, 
                       image_factory=StyledPilImage, 
                       module_drawer=style_modules.get(style)(),
                       color_mask=SolidFillColorMask(back_color = hex_to_rgb(back_color), 
                                                     front_color = hex_to_rgb(fill_color))
)

corner_radius = 20

border_width = 20  # Adjust this to your desired border width
square_img = Image.new("RGBA", (qr_img.size[0] + border_width, qr_img.size[1] + border_width), (0, 0, 0, 0))
draw = ImageDraw.Draw(square_img)
draw.rounded_rectangle([(0, 0), (square_img.size[0], square_img.size[1])], corner_radius,
               fill=hex_to_rgb(fill_color))

# Calculate the position to paste the QR code within the square
qr_pos_x = (square_img.size[0] - qr_img.size[0] ) // 2
qr_pos_y = (square_img.size[1] - qr_img.size[1] ) // 2
qr_mask = qr_img.convert("L")

# Paste the QR code onto the square image
square_img.paste(qr_img, (qr_pos_x, qr_pos_y))



if path.exists(logo_path):
    # Load logo and resize it
    logo = Image.open(logo_path).convert("RGBA")
    logo = logo.resize((95, 95))


    logo_bg = Image.new("RGBA", (120, 120), hex_to_rgb(fill_color))
    draw = ImageDraw.Draw(logo_bg)
    draw.rounded_rectangle([(0, 0), (120, 120)], corner_radius, fill=hex_to_rgb(back_color))

    logo_pos_x = (logo_bg.size[0] - logo.size[0]) // 2
    logo_pos_y = (logo_bg.size[1] - logo.size[1]) // 2


    logo_bg.paste(logo, (logo_pos_x, logo_pos_y), logo)

    # Calculate the position to add the logo
    pos_x = (qr_img.size[0] - logo_bg.size[0]) // 2
    pos_y = (qr_img.size[1] - logo_bg.size[1]) // 2

    # Paste the logo onto the QR code
    qr_img.paste(logo_bg, (pos_x, pos_y))

# Save the modified QR code
qr_img.save(output_folder / 'qr.png')